In [1]:
import torch
import numpy
from ml_modules import *
from sequence_modules import *
from llm_modules import *
from agent import *
from simulation import *

from utils import *

from ml_ops_utils import *

import gc

from hand_strength import *

/home/sachdved/miniconda3/envs/local_llm_host/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!nvidia-smi

Sun Feb 22 17:27:44 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.119.02             Driver Version: 580.119.02     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        Off |   00000000:01:00.0  On |                  N/A |
|  0%   27C    P3             47W /  450W |     305MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
num_players = 2
small_blind = 1
big_blind = 2
num_streets = 4
starting_stack_sizes = 400
max_seq_len = 1024
embed_dim = 256
device = "cuda"


softmax_prob = torch.nn.Softmax(dim=-1)

action_validator = PokerActionValidator(
    num_players = num_players,
    small_blind = small_blind,
    big_blind = big_blind,
    starting_stack_sizes = starting_stack_sizes
)

street_embedder_p1 = StreetPositionalEncoding(
    num_streets = num_streets,
    embedding_dim = embed_dim,
    max_seq_len = max_seq_len,
    device = device
)

table_position_embedder_p1 = TablePositionalEncoding(
    num_players = num_players,
    embedding_dim = embed_dim,
    max_seq_len = max_seq_len,
    device = device
)

action_embedder_p1 = ActionEncoding(
    #num_actions = 21,
    embedding_dim = embed_dim,
    max_seq_len = max_seq_len,
    device = device
)

pot_size_embedder_p1 = PotSizeSequenceEmbedder(
    max_seq_len = max_seq_len,
    pad_value = -1,
    device = device
)

poker_sequence_embedder_p1 = PokerSequenceEmbedder(
    street_input_dimension = embed_dim,
    table_position_input_dimension = embed_dim,
    action_input_dimension = embed_dim,
    latent_dimensions = [embed_dim, embed_dim * 2, embed_dim * (2**2), embed_dim * (2**3)],
    device = device
)
cards_p1 = Cards(device = device)

self_position_embedder_p1 = SelfPositionEmbedder(number_of_positions = num_players, device = device)


street_embedder_p2 = StreetPositionalEncoding(
    num_streets = num_streets,
    embedding_dim = embed_dim,
    max_seq_len = max_seq_len,
    device = "cuda"
)

table_position_embedder_p2 = TablePositionalEncoding(
    num_players = num_players,
    embedding_dim = embed_dim,
    max_seq_len = max_seq_len,
    device = device
)

action_embedder_p2 = ActionEncoding(
    #num_actions = 21,
    embedding_dim = embed_dim,
    max_seq_len = max_seq_len,
    device = device
)

pot_size_embedder_p2 = PotSizeSequenceEmbedder(
    max_seq_len = max_seq_len,
    pad_value = -1,
    device = device
)

poker_sequence_embedder_p2 = PokerSequenceEmbedder(
    street_input_dimension = embed_dim,
    table_position_input_dimension = embed_dim,
    action_input_dimension = embed_dim,
    latent_dimensions = [embed_dim, embed_dim * 2, embed_dim * (2**2), embed_dim * (2**3)],
    device = device
)

cards_p2 = Cards(device = device)

self_position_embedder_p2 = SelfPositionEmbedder(number_of_positions = num_players, device = device)

deck_order_shuffled = torch.argsort(torch.rand(1,52))

In [4]:
!nvidia-smi

Sun Feb 22 17:27:44 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.119.02             Driver Version: 580.119.02     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        Off |   00000000:01:00.0  On |                  N/A |
|  0%   28C    P1             51W /  450W |     899MiB /  32607MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
model_name = "./models/qwen3-1point7b/"

tokenizer, model = load_model(model_name)

model

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.87it/s]
The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layer

In [6]:
policy_model_p1 = PolicyModel(
    num_players = num_players,
    self_position_embedder = self_position_embedder_p1,
    active_players_hidden_dims = [1024, 2048],
    stack_size_hidden_dims = [1024, 2048],
    card_embeddings_hidden_dims = [2048, 2048],
    final_output_hidden_dims = [1024, 512, 256],
    value_output_hidden_dims = [1024, 512, 256],
    dropout_rate = 0,
    device = 'cuda',
)

policy_model_p2 = PolicyModel(
    num_players = num_players,
    self_position_embedder = self_position_embedder_p2,
    active_players_hidden_dims = [1024, 2048],
    stack_size_hidden_dims = [1024, 2048],
    card_embeddings_hidden_dims = [2048, 2048],
    final_output_hidden_dims = [1024, 512, 256],
    value_output_hidden_dims = [1024, 512, 256],
    dropout_rate = 0,
    device = 'cuda',
)

poker_player_p1 = PokerAgent(
    cards_p1,
    street_embedder_p1,
    table_position_embedder_p1,
    action_embedder_p1,
    pot_size_embedder_p1,
    poker_sequence_embedder_p1,
    model,
    policy_model_p1,
    device = 'cuda',
    llm_train = False
)

poker_player_p2 = PokerAgent(
    cards_p2,
    street_embedder_p2,
    table_position_embedder_p2,
    action_embedder_p2,
    pot_size_embedder_p2,
    poker_sequence_embedder_p2,
    model,
    policy_model_p2,
    device = 'cuda',
    llm_train = False
)

In [26]:
batch_size = 1024

deck_order_shuffled = torch.argsort(torch.rand(1,52))


street_idxs = (torch.zeros((batch_size, 1024)) + 6).long()
street_idxs[:, :2] = 0 # posting small blind and big blind

table_position_idxs = (torch.zeros((batch_size, 1024)) + 2).long()
table_position_idxs[:, 0] = 0 # sb/b
table_position_idxs[:, 1] = 1 # bb

action_idxs = (torch.zeros((batch_size, 1024)) + 21).long()
action_idxs[:, 0] = 0 # post sb
action_idxs[:, 1] = 1 # post bb

pot_size_sequence = (torch.zeros((batch_size, 1024)) - 1)
pot_size_sequence[:, 0] = 1
pot_size_sequence[:, 1] = 3


active_players = torch.Tensor([
    [1, 1]
]).tile(batch_size, 1)
print(active_players.shape)
stack_size = torch.Tensor(
    [
        [399, 398],
    ]
).tile(batch_size, 1)
stack_size.shape

sb_cards = torch.zeros((batch_size,2,7)).to(dtype=torch.long).to('cuda')
bb_cards = torch.zeros((batch_size,2,7)).to(dtype=torch.long).to('cuda')

sb_cards[:, 0, :2] = deck_order_shuffled[0, :2]%13
sb_cards[:, 1, :2] = deck_order_shuffled[0, :2]//13

sb_cards[:, 0, 2:] = 13
sb_cards[:, 1, 2:] = 4


bb_cards[:, 0, :2] = deck_order_shuffled[0, 2:4]%13
bb_cards[:, 1, :2] = deck_order_shuffled[0, 2:4]//13

bb_cards[:, 0, 2:] = 13
bb_cards[:, 1, 2:] = 4

p1_position_player = torch.Tensor([0]).to('cuda').tile((batch_size))
p2_position_player = torch.Tensor([1]).to('cuda').tile((batch_size))

table = {
    0 : [poker_player_p1, sb_cards, p1_position_player],
    1 : [poker_player_p2, bb_cards, p2_position_player]
}

(
    sim_street_idxs, 
    sim_table_position_idxs, 
    sim_action_idxs, 
    sim_pot_size_sequence, 
    sim_active_players, 
    sim_stack_size,
    sim_table
) = simulate_hand(
    num_players,
    street_idxs,
    table_position_idxs,
    action_idxs,
    pot_size_sequence,
    active_players,
    stack_size,
    table,
    action_validator,
    deck_order_shuffled,
)

torch.Size([1024, 2])
after action 2 


2
1
tensor([0, 1])
tensor([1., 3.])
tensor([0, 1])
tensor([0, 0])
tensor([399., 398.])
tensor([1., 1.])

 

0 is the next to act
4
tensor(399.)
after action 3 


3
2
tensor([ 0,  1, 15])
tensor([  1.,   3., 255.])
tensor([0, 1, 0])
tensor([0, 0, 0])
tensor([147., 398.])
tensor([1., 1.])

 

1 is the next to act
tensor(504.)
tensor(398.)
after action 4 


4
3
tensor([ 0,  1, 15, 16])
tensor([  1.,   3., 255., 653.])
tensor([0, 1, 0, 1])
tensor([0, 0, 0, 0])
tensor([147.,   0.])
tensor([1., 1.])

 

0 is the next to act
after action 5 


5
4
tensor([ 0,  1, 15, 16,  2])
tensor([  1.,   3., 255., 653., 653.])
tensor([0, 1, 0, 1, 0])
tensor([0, 0, 0, 0, 0])
tensor([147.,   0.])
tensor([0., 1.])

 

-1 is the next to act


In [27]:
from hand_strength import *

In [28]:
rewards = determine_winner(
    sim_active_players[-1],
    sim_pot_size_sequence[-1],
    sim_stack_size[-1],
    sim_table
)

In [29]:
sim_table[1][1][-1]

tensor([[ 2,  1, 13, 13, 13, 13, 13],
        [ 0,  3,  4,  4,  4,  4,  4]], device='cuda:0')

In [30]:
sim_table[0][1][-1]

tensor([[ 7,  3, 13, 13, 13, 13, 13],
        [ 3,  3,  4,  4,  4,  4,  4]], device='cuda:0')

In [31]:
sim_action_idxs[-1][:10]

tensor([ 0,  1, 15, 16,  2, 20, 21, 21, 21, 21])

In [32]:
rewards

tensor([-253.,  253.])

In [33]:
for i in range(num_players):

    sim_table[i].append(rewards[i])


In [34]:
sim_table[0][0]

PokerAgent(
  (cards): Cards(
    (rank_embedder): Embedding(14, 1024)
    (suit_embedder): Embedding(5, 1024)
  )
  (street_embedder): StreetPositionalEncoding(
    (street_embedder): Embedding(7, 256)
  )
  (table_position_embedder): TablePositionalEncoding(
    (player_embedder): Embedding(4, 256)
  )
  (action_embedder): ActionEncoding(
    (action_embedder): Embedding(22, 256)
  )
  (pot_size_embedder): PotSizeSequenceEmbedder()
  (poker_sequence_embedder): PokerSequenceEmbedder(
    (street_MLP): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (2): ReLU()
      (3): Linear(in_features=256, out_features=512, bias=True)
      (4): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (5): ReLU()
      (6): Linear(in_features=512, out_features=1024, bias=True)
      (7): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (8): ReLU()
      (9): Linear(in_features=1024, o

In [37]:
batch_indices = torch.where(sim_table_position_idxs[-1] == 0)[0]

In [43]:
sim_table_position_idxs[:, :7]

tensor([[0, 1, 2, 2, 2, 2, 2],
        [0, 1, 0, 2, 2, 2, 2],
        [0, 1, 0, 1, 2, 2, 2],
        [0, 1, 0, 1, 0, 2, 2],
        [0, 1, 0, 1, 0, 2, 2],
        [0, 1, 0, 1, 0, 2, 2]])

In [39]:
sim_street_idxs[batch_indices][:, :7]

tensor([[0, 0, 6, 6, 6, 6, 6],
        [0, 0, 0, 0, 6, 6, 6],
        [0, 0, 0, 0, 0, 5, 6]])

In [40]:
sim_action_idxs[:5, :10]

tensor([[ 0,  1, 21, 21, 21, 21, 21, 21, 21, 21],
        [ 0,  1, 15, 21, 21, 21, 21, 21, 21, 21],
        [ 0,  1, 15, 16, 21, 21, 21, 21, 21, 21],
        [ 0,  1, 15, 16,  2, 21, 21, 21, 21, 21],
        [ 0,  1, 15, 16,  2, 20, 21, 21, 21, 21]])

In [47]:
torch.where(sim_table_position_idxs[-1] == 0)[0]

tensor([0, 2, 4])

In [49]:
sim_table_position_idxs[-1, :10]

tensor([0, 1, 0, 1, 0, 2, 2, 2, 2, 2])

In [53]:
p0_action_batch_indices = torch.where(
    action_validator.get_next_to_act(
        sim_street_idxs,
        sim_table_position_idxs,
        sim_action_idxs,
        sim_active_players
    ) == 0
)[0]

In [54]:
p0_action_batch_indices

tensor([0, 2])

In [80]:
sim_action_idxs[:, :10]

tensor([[ 0,  1, 21, 21, 21, 21, 21, 21, 21, 21],
        [ 0,  1, 15, 21, 21, 21, 21, 21, 21, 21],
        [ 0,  1, 15, 16, 21, 21, 21, 21, 21, 21],
        [ 0,  1, 15, 16,  2, 21, 21, 21, 21, 21],
        [ 0,  1, 15, 16,  2, 20, 21, 21, 21, 21],
        [ 0,  1, 15, 16,  2, 20, 21, 21, 21, 21]])

In [83]:
sim_action_idxs[p0_action_batch_indices + 1][:, :10]

tensor([[ 0,  1, 15, 21, 21, 21, 21, 21, 21, 21],
        [ 0,  1, 15, 16,  2, 21, 21, 21, 21, 21]])

In [84]:
mask = (sim_action_idxs[p0_action_batch_indices + 1][:, :10] == 21)

In [87]:
mask.float().argmax(dim = 1)

tensor([3, 5])

In [90]:
sim_action_idxs[p0_action_batch_indices+1, mask.float().argmax(dim = 1) - 1]

tensor([15,  2])

In [52]:
sim_action_idxs[-1][:10]

tensor([ 0,  1, 15, 16,  2, 20, 21, 21, 21, 21])

In [55]:
position = sim_table[0][2][p0_action_batch_indices]
cards = sim_table[0][1][p0_action_batch_indices]
street_idxs = sim_street_idxs[p0_action_batch_indices]
table_position_idxs = sim_table_position_idxs[p0_action_batch_indices]
action_idxs = sim_action_idxs[p0_action_batch_indices]
pot_size_sequence = sim_pot_size_sequence[p0_action_batch_indices]
active_players = sim_active_players[p0_action_batch_indices]
stack_size = sim_stack_size[p0_action_batch_indices]


In [59]:
street_idxs[:, :10]

tensor([[0, 0, 6, 6, 6, 6, 6, 6, 6, 6],
        [0, 0, 0, 0, 6, 6, 6, 6, 6, 6]])

In [69]:
sim_stack_size[[0,2]]

tensor([[399., 398.],
        [147.,   0.]])

In [61]:
sim_stack_size[:, :10]

tensor([[399., 398.],
        [147., 398.],
        [147.,   0.],
        [147.,   0.],
        [147.,   0.],
        [147.,   0.]])

In [72]:
outputs = sim_table[0][0](
                position,
                cards,
                street_idxs,
                table_position_idxs,
                action_idxs,
                pot_size_sequence,
                active_players.to('cuda'),
                stack_size.to('cuda')
            )

In [74]:
outputs['probits']

tensor([[-4.5540e-01, -7.9507e-01,  2.0691e-01,  6.9841e-01,  5.8775e-01,
         -3.6335e-01, -5.9704e-01, -2.0490e-01,  4.9412e-01,  9.1883e-01,
          7.1984e-03,  8.3491e-02,  6.8163e-01,  1.2774e+00, -4.8085e-01,
         -3.1557e-01,  1.4568e+00, -3.0666e-01,  4.9429e-01, -6.2912e-01,
          8.7670e-02],
        [-1.1607e-01, -1.3605e-01, -6.8385e-02,  1.5207e-01,  1.4174e-01,
         -1.4663e-01, -2.9819e-01, -8.9445e-02,  9.8292e-04,  4.4968e-01,
          8.7866e-02, -8.2074e-02,  2.2530e-01,  3.5804e-01, -6.4547e-02,
         -3.3304e-01,  4.5869e-01, -3.2877e-01, -1.8781e-03, -1.6148e-01,
         -2.3242e-02]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [95]:
neg_log_probs = outputs['probits'][torch.arange(outputs['probits'].shape[0]), sim_action_idxs[p0_action_batch_indices+1, mask.float().argmax(dim = 1) - 1]]

In [96]:
sim_table[0][-1]

tensor([[-253.],
        [-253.],
        [-253.],
        [-253.],
        [-253.],
        [-253.]])

In [98]:
loss = (neg_log_probs * sim_table[0][-1][0][0]).mean()

In [120]:
sim_action_idxs[0][:10]

tensor([ 0,  1, 21, 21, 21, 21, 21, 21, 21, 21])

In [122]:
sim_action_idxs[1][:10]

tensor([ 0,  1, 14, 21, 21, 21, 21, 21, 21, 21])

In [121]:
sim_action_idxs[2][:10]

tensor([ 0,  1, 14,  8, 21, 21, 21, 21, 21, 21])

In [124]:
sim_action_idxs[6][:10]

tensor([ 0,  1, 14,  8, 16,  2, 20, 21, 21, 21])

In [128]:
sim_table_position_idxs[6, :10]

tensor([0, 1, 0, 1, 0, 1, 2, 2, 2, 2])

In [136]:
sim_action_idxs[torch.where(sim_table_position_idxs[-1][2:] == 0)[0]][:,:7]

tensor([[ 0,  1, 21, 21, 21, 21, 21],
        [ 0,  1, 14,  8, 21, 21, 21]])

In [137]:
sim_table_position_idxs[torch.where(sim_table_position_idxs[-1][2:] == 0)[0]][:,:7]

tensor([[0, 1, 2, 2, 2, 2, 2],
        [0, 1, 0, 1, 2, 2, 2]])

In [53]:
sim_table[0]

[PokerAgent(
   (cards): Cards(
     (rank_embedder): Embedding(14, 1024)
     (suit_embedder): Embedding(5, 1024)
   )
   (street_embedder): StreetPositionalEncoding(
     (street_embedder): Embedding(7, 256)
   )
   (table_position_embedder): TablePositionalEncoding(
     (player_embedder): Embedding(4, 256)
   )
   (action_embedder): ActionEncoding(
     (action_embedder): Embedding(22, 256)
   )
   (pot_size_embedder): PotSizeSequenceEmbedder()
   (poker_sequence_embedder): PokerSequenceEmbedder(
     (street_MLP): Sequential(
       (0): Linear(in_features=256, out_features=256, bias=True)
       (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
       (2): ReLU()
       (3): Linear(in_features=256, out_features=512, bias=True)
       (4): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
       (5): ReLU()
       (6): Linear(in_features=512, out_features=1024, bias=True)
       (7): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
       (8): ReLU()
       (9):

In [27]:
sim_table[1][1][-1]

tensor([[ 3,  4, 11,  8,  4,  2,  5],
        [ 2,  0,  1,  2,  2,  1,  1]], device='cuda:0')

In [ ]:
  num_players = 6                                                                                                                                                                                              
  small_blind = 1                                                                                                                                                                                              
  big_blind = 2                                                                                                                                                                                                
  num_streets = 4                                                                                                                                                                                              
  starting_stack_sizes = 400                                                                                                                                                                                   
  max_seq_len = 1024
  embed_dim = 256
  device = "cuda"

  # Shared configuration
  softmax_prob = torch.nn.Softmax(dim=-1)

  # Create a dictionary to store all player components
  poker_components = {}

  # Initialize shared components
  poker_components['cards'] = Cards(device=device)
  poker_components['deck_order'] = torch.argsort(torch.rand(1, 52))

  # Initialize each player's components
  for player_id in range(num_players):
      poker_components[f'p{player_id}'] = {
          'street_embedder': StreetPositionalEncoding(
              num_streets=num_streets,
              embedding_dim=embed_dim,
              max_seq_len=max_seq_len,
              device=device
          ),
          'table_position_embedder': TablePositionalEncoding(
              num_players=num_players,
              embedding_dim=embed_dim,
              max_seq_len=max_seq_len,
              device=device
          ),
          'action_embedder': ActionEncoding(
              embedding_dim=embed_dim,
              max_seq_len=max_seq_len,
              device=device
          ),
          'pot_size_embedder': PotSizeSequenceEmbedder(
              max_seq_len=max_seq_len,
              pad_value=-1,
              device=device
          ),
          'poker_sequence_embedder': PokerSequenceEmbedder(
              street_input_dimension=embed_dim,
              table_position_input_dimension=embed_dim,
              action_input_dimension=embed_dim,
              latent_dimensions=[embed_dim, embed_dim * 2, embed_dim * (2**2), embed_dim * (2**3)],
              device=device
          ),
          'self_position_embedder': SelfPositionEmbedder(
              number_of_positions=num_players,
              device=device
          )
      }